In [1]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
import base64
from bs4 import BeautifulSoup as bs
import re

In [2]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
import base64
from bs4 import BeautifulSoup as bs

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
with open('token.json', 'rb') as token:
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

service = build('gmail', 'v1', credentials=creds)
profile = service.users().getProfile(userId='me').execute()
profile

{'emailAddress': 'dongoodygoody@gmail.com',
 'messagesTotal': 3798,
 'threadsTotal': 3066,
 'historyId': '2835120'}

In [5]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
with open('token.json', 'rb') as token:
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

service = build('gmail', 'v1', credentials=creds)

In [24]:
filter = service.users().messages().list(userId = 'me', maxResults=20,
                                         q = 'from:no_reply@accessbankplc.com \
                                             subject:AccessAlert Transaction Alert').execute()

filter_id = filter['messages']
id_lst = []
for ids in filter_id:
    id_lst.append(ids['id'])
    

In [32]:
snippet_lst = []
for each_id in id_lst:
    msgs = service.users().messages().get(userId = 'me', id = each_id).execute()
    snippet = msgs['snippet']
    snippet_lst.append(snippet)

print(len(snippet_lst))

20


In [33]:
snippet_lst

['Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 300.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 162******608 Account Name',
 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 300.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name',
 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 2100.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name',
 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 5021.50 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 162******608 Account Name',
 'Dear GOODRICH IFEANYI OKORO, Your account has been Credited NGN 5000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name',
 

In [6]:
result = service.users().messages().list(userId='me').execute()
messages = result.get('messages')
msg = messages[:3]
msg

[{'id': '18304d6779c2d13c', 'threadId': '18304d6779c2d13c'},
 {'id': '18304c8e5075a81a', 'threadId': '18304c8e5075a81a'},
 {'id': '1830481aa1352a24', 'threadId': '1830481aa1352a24'}]

In [11]:
id_lst = []
for ids in msg:
    id_lst.append(ids['id'])
txt = service.users().messages().get(userId='me', id='182fa3a277f6fd04').execute()
txt

{'id': '182fa3a277f6fd04',
 'threadId': '182fa3a277f6fd04',
 'labelIds': ['CATEGORY_UPDATES', 'INBOX'],
 'snippet': 'We noticed a login to your account @GoodrichOkoro from a new device. Was this you? New login Location* Nigeria Device Chrome on Linux *Location is approximate based on the login&#39;s IP address. If',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'dongoodygoody@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:6f02:e:b0:23:4722:82d with SMTP id 14csp288701rcj;        Thu, 1 Sep 2022 11:04:51 -0700 (PDT)'},
   {'name': 'X-Google-Smtp-Source',
    'value': 'AA6agR6gFhT6SvFW3SMW+y7csq7Nz9Ntj8FUFeBjybzm1GjxsvVT9TTVegz9zwo90bpYmfPuCbMl'},
   {'name': 'X-Received',
    'value': 'by 2002:a25:f80c:0:b0:691:7d23:74c9 with SMTP id u12-20020a25f80c000000b006917d2374c9mr21247003ybd.112.1662055491574;        Thu, 01 Sep 2022 11:04:51 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': '

In [7]:
id_lst = []
msg_lst = []
for ids in msg:
    id_lst.append(ids['id'])
    
for id in id_lst:
    message = service.users().messages().get(userId='me', id=id).execute()
    # d = ['Debited|Credited']
    pattern = re.compile(r'Debited|Credited')
    snippet = message['snippet']
    if re.search('Debited|Credited', snippet):
        msg_lst.append(snippet)
    else:
        pass

print(msg_lst[0])

IndexError: list index out of range

In [13]:
id_lst = []
msg_lst = []
for ids in msg:
    id_lst.append(ids['id'])
    
for id in id_lst:
    message = service.users().messages().get(userId='me', id=id).execute()
    # d = ['Debited|Credited']
    pattern = re.compile(r'Debited|Credited')
    snippet = message['snippet']
    if re.search('Debited|Credited', snippet):
        pad = re.findall('\d*\.?\d+', snippet)
        print(pad)
    else:
        pass

['5000.00', '162', '608']
['5000.00', '070', '357']
['10000.00', '162', '608']
['33000.00', '162', '608']


In [103]:
msg_lst

['Dear GOODRICH IFEANYI OKORO, Your account has been Credited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name',
 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 162******608 Account Name',
 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3100.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name',
 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 2000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name',
 'Dear GOODRICH IFEANYI OKORO, Your account has been Credited NGN 1000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name'

In [105]:
pat = re.compile(r'\d*\.?\d+')
for x in msg_lst:
   pad = re.findall('\d*\.?\d+', x)

print(pad[0])

11200.00


In [68]:
txt1 = service.users().messages().get(userId='me', id='182ef29b75d68c6b').execute()
txt1

{'id': '182ef29b75d68c6b',
 'threadId': '182ef29b75d68c6b',
 'labelIds': ['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX'],
 'snippet': 'Dear GOODRICH IFEANYI OKORO, Your account has been Credited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name',
 'payload': {'partId': '',
  'mimeType': 'text/html',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'dongoodygoody@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:6f02:2b:b0:22:a77d:eece with SMTP id 43csp2220383rcj;        Tue, 30 Aug 2022 07:31:04 -0700 (PDT)'},
   {'name': 'X-Google-Smtp-Source',
    'value': 'AA6agR7qsUn4ZJ7nbOoEyakm2DAemcV/Wf42owLDiei7usupLLNTy2mPMeQzB3xzsC5E0FiU0d0d'},
   {'name': 'X-Received',
    'value': 'by 2002:ac8:5f12:0:b0:343:6510:ed6f with SMTP id x18-20020ac85f12000000b003436510ed6fmr14881218qta.342.1661869864527;        Tue, 30 Aug 2022 07:31:04 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value'

In [69]:
txt2 = service.users().messages().get(userId='me', id='182ef29b4506246f').execute()
txt2

{'id': '182ef29b4506246f',
 'threadId': '182ef29b4506246f',
 'labelIds': ['CATEGORY_UPDATES', 'INBOX'],
 'snippet': 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 162******608 Account Name',
 'payload': {'partId': '',
  'mimeType': 'text/html',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'dongoodygoody@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:6f02:2b:b0:22:a77d:eece with SMTP id 43csp2220382rcj;        Tue, 30 Aug 2022 07:31:04 -0700 (PDT)'},
   {'name': 'X-Google-Smtp-Source',
    'value': 'AA6agR67qsAvrXSWUU4XRBuv+6BhrCfzfKUsl8fIesf3VQGxGUnMxJnviP5IlCoMsZx0sdSr/xvK'},
   {'name': 'X-Received',
    'value': 'by 2002:a05:622a:1185:b0:343:7de8:c0f2 with SMTP id m5-20020a05622a118500b003437de8c0f2mr14578169qtk.256.1661869864402;        Tue, 30 Aug 2022 07:31:04 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rs

In [70]:
txt3 = service.users().messages().get(userId='me', id='182ef29a5825df86').execute()
txt3

{'id': '182ef29a5825df86',
 'threadId': '182ef29a5825df86',
 'labelIds': ['CATEGORY_UPDATES', 'INBOX'],
 'snippet': 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3100.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name',
 'payload': {'partId': '',
  'mimeType': 'text/html',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'dongoodygoody@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:6f02:2b:b0:22:a77d:eece with SMTP id 43csp2220343rcj;        Tue, 30 Aug 2022 07:31:00 -0700 (PDT)'},
   {'name': 'X-Google-Smtp-Source',
    'value': 'AA6agR4ymFeNgXMtmUGbMz8qvjNTSWpUJGaiwJESTeiSPVk2tQj46/xWA2/bB59/u2hmyzmDC74u'},
   {'name': 'X-Received',
    'value': 'by 2002:a05:622a:608:b0:344:a856:c24 with SMTP id z8-20020a05622a060800b00344a8560c24mr14858487qta.678.1661869860323;        Tue, 30 Aug 2022 07:31:00 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-

In [71]:
txt4 = service.users().messages().get(userId='me', id='182eee8583356d60').execute()
txt4

{'id': '182eee8583356d60',
 'threadId': '182eee8583356d60',
 'labelIds': ['CATEGORY_PROMOTIONS', 'INBOX'],
 'snippet': 'Save up to $300 on PCs.\u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'dongoodygoody@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:6f02:2b:b0:22:a77d:eec

In [73]:
txt5 = service.users().messages().get(userId='me', id='182ef2d876dc2625').execute()
txt5

{'id': '182ef2d876dc2625',
 'threadId': '182ef2d876dc2625',
 'labelIds': ['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX'],
 'snippet': 'Congratulations to Costa and You! If you cannot see the email below, please click here to see the web version. Dear FreeBitco.in User, You are receiving this email because you are an active user at',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'dongoodygoody@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:6f02:2b:b0:22:a77d:eece with SMTP id 43csp2222754rcj;        Tue, 30 Aug 2022 07:35:15 -0700 (PDT)'},
   {'name': 'X-Google-Smtp-Source',
    'value': 'AA6agR4zmMmklPZl31wSPjRdWJ4VkwSZexrr6RBm5zz4ipDRGI4J+5eVmX40TbZXxnnI9krFBPk/'},
   {'name': 'X-Received',
    'value': 'by 2002:a05:620a:2452:b0:6bb:d8ba:ca65 with SMTP id h18-20020a05620a245200b006bbd8baca65mr12104528qkn.263.1661870114758;        Tue, 30 Aug 2022 07:35:14 -0700 (PDT)'},
   {'name': 'ARC-Se

In [47]:
you = txt5['snippet']
you

'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 2000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name'

In [48]:
import re
bank = []
pattern = re.compile(r'Debited')
b = you if re.search('Debited', you) else 3
print(b)

# import re
# pattern=re.compile(r'ack') 
# str_match = [x for x in my_list if re.search('ack', x)]
# print(str_match)

Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 2000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name


In [52]:
ids = [txt1, txt2, txt3, txt4, txt5]
pattern = re.compile(r'Debited|Credited')
for txt in ids:
    snippet = txt['snippet']
    if re.search('Debited|Credited', snippet):
        print(snippet)
    else:
        print('None')

Dear GOODRICH IFEANYI OKORO, Your account has been Credited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name
Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 162******608 Account Name
Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3100.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name
None
Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 2000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name


In [53]:
ids = [txt1, txt2, txt3, txt4, txt5]
d = 'Debited|Credited'
pattern = re.compile(r'd')
for txt in ids:
    snippet = txt['snippet']
    if re.search('d', snippet):
        print(snippet)
    else:
        print('None')

Dear GOODRICH IFEANYI OKORO, Your account has been Credited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name
Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 162******608 Account Name
Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3100.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name
None
Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 2000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name


In [27]:
payload = txt1['payload']['headers']
payload

[{'name': 'Delivered-To', 'value': 'dongoodygoody@gmail.com'},
 {'name': 'Received',
  'value': 'by 2002:a05:6f02:2b:b0:22:a77d:eece with SMTP id 43csp2220382rcj;        Tue, 30 Aug 2022 07:31:04 -0700 (PDT)'},
 {'name': 'X-Google-Smtp-Source',
  'value': 'AA6agR67qsAvrXSWUU4XRBuv+6BhrCfzfKUsl8fIesf3VQGxGUnMxJnviP5IlCoMsZx0sdSr/xvK'},
 {'name': 'X-Received',
  'value': 'by 2002:a05:622a:1185:b0:343:7de8:c0f2 with SMTP id m5-20020a05622a118500b003437de8c0f2mr14578169qtk.256.1661869864402;        Tue, 30 Aug 2022 07:31:04 -0700 (PDT)'},
 {'name': 'ARC-Seal',
  'value': 'i=1; a=rsa-sha256; t=1661869864; cv=none;        d=google.com; s=arc-20160816;        b=WFqrEmr/PaXscWQFoKGOgmoodOz0c9sYTNuNXehIRDfkfBBAouEzy4JMYzLKCq6oDX         wmV2COEcoOIlQKNJOyTNNVw83C8mGY8XP+BkcNj7Hax7i56SjOHcE3pxuItRRjR0KUrq         8d/ijKjYvfL2vX6GcYNNQZgPt0M4p69CoUI254GAXHH30aP3XIkw63K2LO+EVwKrTgl3         Rzte2/Uxbl2ZvRZ8ranlK17GrvvmJAL0uZE7jtRySSN6yQiv1avzZG7MLbs/oGI9ASDi         7nyIXBA+5drhcix12cXIj5/b8NNSDXD

In [28]:
we = [d for d in payload if d['name'] == 'Subject']
we

[{'name': 'Subject',
  'value': 'AccessAlert Transaction Alert [Debit: 3,000.00 NGN]'}]

In [21]:
payload = txt['payload']
headers = payload['headers']

parts = payload.get('parts')[0]
data = parts['body']['data']
data = data.replace('-','+').replace('_','/')
decode_data = base64.b64decode(data)
decode_data

b'Top stories for Goodrich\r\n\r\n-----\r\n\r\nQuestion: How long should it take to learn Python completely and start developing programs?\r\n\r\n\r\nAnswer from Larry Lock\r\n\r\n\r\nI\xe2\x80\x99m a professional programmer (30+ years experience).\r\n\r\nHere are some hints, in addition to an answer to your question:\r\n\r\n1. Most professional programmers never learn a prog \r\nRead More: https://www.quora.com/qemail/tc?al_imp=eyJ0eXBlIjogMzMsICJoYXNoIjogIjE2Nzk1NjAwNDEyOTQwMjczNjd8MXwxfDU1MTYyMjc4In0%3D&al_pri=8&aoid=qopuVW6VbFH&aoty=1&aty=4&cp=1&et=2&id=9f9d2dd44f184eb79b0eee6ba9bdb014&uid=m0UWbQjebdu\r\n-----\r\nQuestion: Why is C++ getting replaced by Rust while C is not?\r\n\r\n\r\nAnswer from Ian Joyner\r\n\r\n\r\nBoth C and C++ should be replaced as system languages. And they should not be used for problem-oriented programming in the first place. Yes, C++ tries to be a language for \r\nRead More: https://www.quora.com/qemail/tc?al_imp=eyJ0eXBlIjogMzMsICJoYXNoIjogIjE2Nzk1NjAwND

In [24]:
soup = bs(decode_data,"lxml" )
body = soup.body()
body

<p>Top stories for Goodrich

-----

Question: How long should it take to learn Python completely and start developing programs?


Answer from Larry Lock


I’m a professional programmer (30+ years experience).

Here are some hints, in addition to an answer to your question:

1. Most professional programmers never learn a prog 
Read More: https://www.quora.com/qemail/tc?al_imp=eyJ0eXBlIjogMzMsICJoYXNoIjogIjE2Nzk1NjAwNDEyOTQwMjczNjd8MXwxfDU1MTYyMjc4In0%3D&amp;al_pri=8&amp;aoid=qopuVW6VbFH&amp;aoty=1&amp;aty=4&amp;cp=1&amp;et=2&amp;id=9f9d2dd44f184eb79b0eee6ba9bdb014&amp;uid=m0UWbQjebdu
-----
Question: Why is C++ getting replaced by Rust while C is not?


Answer from Ian Joyner


Both C and C++ should be replaced as system languages. And they should not be used for problem-oriented programming in the first place. Yes, C++ tries to be a language for 
Read More: https://www.quora.com/qemail/tc?al_imp=eyJ0eXBlIjogMzMsICJoYXNoIjogIjE2Nzk1NjAwNDEyOTQwMjczNjd8MnwxfDMyNzU1MTkzNyJ9&amp;al_pri=8&am